In [48]:
pip install --upgrade qiskit

   ---------------------------------------- 0.0/4.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.6 MB 653.6 kB/s eta 0:00:07
   ---------------------------------------- 0.0/4.6 MB 653.6 kB/s eta 0:00:07
    --------------------------------------- 0.1/4.6 MB 731.4 kB/s eta 0:00:07
   - -------------------------------------- 0.1/4.6 MB 711.9 kB/s eta 0:00:07
   -- ------------------------------------- 0.2/4.6 MB 962.7 kB/s eta 0:00:05
   -- ------------------------------------- 0.3/4.6 MB 983.0 kB/s eta 0:00:05
   -- ------------------------------------- 0.3/4.6 MB 936.6 kB/s eta 0:00:05
   --- ------------------------------------ 0.4/4.6 MB 1.1 MB/s eta 0:00:04
   ---- ----------------------------------- 0.5/4.6 MB 1.2 MB/s eta 0:00:04
   ----- ---------------------------------- 0.6/4.6 MB 1.3 MB/s eta 0:00:04
   ------ --------------------------------- 0.7/4.6 MB 1.4 MB/s eta 0:00:03
   ------- -------------------------------- 0.9/4.6 MB 1.5 MB/s eta 0:00:03
   --

  You can safely remove it manually.
  You can safely remove it manually.


# Classe Sampler e Estimator

In [51]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_bloch_multivector
import qiskit.quantum_info as qi
from qiskit.quantum_info import Operator, Pauli
from qiskit.primitives import Sampler, Estimator
import numpy as np
from qiskit.circuit.library import RXGate

### Passo 1 - Criar o Circuito Quântico de 1 qubit:

In [52]:
# Criando o circuito quântico para preparar o estado conforme descrito
qc = QuantumCircuit(1)
qc.rx(np.pi / 3, 0)
qc.rz(np.pi / 4, 0)

# Criando o simulador
simulator = AerSimulator()

# Criando o sampler
sampler = Sampler(backend=simulator)

# Função para calcular valor esperado de uma base específica
def calcular_valor_esperado(circuito, sampler, base=None):
    if base == 'X':
        circuito.h(0)  # Aplica a porta Hadamard para medir na base X
    elif base == 'Y':
        circuito.rx(-np.pi / 2, 0)  # Aplica a rotação RX para medir na base Y
    
    circuito.measure_all()
    
    transpiled_circuit = transpile(circuito, simulator)
    
    # Executando o circuito no sampler
    result = sampler.run(transpiled_circuit, shots=10000).result()
    counts = result.get_counts()

    p_0 = counts.get('0', 0) / 10000
    p_1 = counts.get('1', 0) / 10000
    valor_esperado = p_0 - p_1
    
    return valor_esperado

TypeError: Sampler.__init__() got an unexpected keyword argument 'backend'

### Passo 2 - Calcular o Valor Esperado de Z usando o Sampler:

In [46]:
#Quando base=None é passado para a função calcular_valor_esperado,
#o circuito é medido na base Z diretamente.

# Executando o circuito e calculando o valor esperado para a base Z
valor_esperado_z = calcular_valor_esperado(qc.copy(), simulator, base=None)

# Exibindo o resultado para a base Z
print(f"Valor esperado para a base Z: {valor_esperado_z}")

TypeError: Sampler.__init__() takes 1 positional argument but 2 were given

### Passo 3 - Calcular o Valor Esperado de X usando o Sampler:

In [35]:
# Executando o circuito e calculando o valor esperado para a base X
valor_esperado_x = calcular_valor_esperado(qc.copy(), simulator, base='X')

# Exibindo o resultado para a base X
print(f"Valor esperado para a base X: {valor_esperado_x}")

Valor esperado para a base X: 0.6048


### Passo 4 - Calcular o Valor Esperado de Y usando o Sampler:

In [36]:
# Executando o circuito e calculando o valor esperado para a base Y
valor_esperado_y = calcular_valor_esperado(qc.copy(), simulator, base='Y')

# Exibindo o resultado para a base Y
print(f"Valor esperado para a base Y: {valor_esperado_y}")

Valor esperado para a base Y: 0.6142000000000001


### Passo 5 - Validar os Resultados usando o Estimator:

In [37]:
# Utilizando o Estimator para comparar os resultados
estimator = Estimator()

# Observáveis convertidos para Pauli
observables = [Pauli('Z'), Pauli('X'), Pauli('Y')]
jobs = [estimator.run(qc, obs) for obs in observables]
results = [job.result().values[0] for job in jobs]

# Resultados com o Sampler e o Estimator
print("Resultados usando o Sampler:")
print(f"Valor esperado de Z: {valor_esperado_z}")
print(f"Valor esperado de X: {valor_esperado_x}")
print(f"Valor esperado de Y: {valor_esperado_y}")

print("\nResultados usando o Estimator:")
print(f"Valor esperado de Z: {results[0]}")
print(f"Valor esperado de X: {results[1]}")
print(f"Valor esperado de Y: {results[2]}")


Resultados usando o Sampler:
Valor esperado de Z: 0.5022
Valor esperado de X: 0.6048
Valor esperado de Y: 0.6142000000000001

Resultados usando o Estimator:
Valor esperado de Z: 0.5
Valor esperado de X: 0.6123724356957945
Valor esperado de Y: -0.6123724356957945
